In [1]:
import pandas as pd
import numpy as np
import json
import glob
import os

OUT_DIR = './data/processed/lol'
OUT_EV = f'{OUT_DIR}/events.csv'
OUT_DESC = f'{OUT_DIR}/desc.json'
IN_DIR = './data/raw/lol'

all_files = glob.glob(os.path.join(IN_DIR, '*.csv'))
df = (pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
      .assign(ts = lambda _d: (pd.to_datetime(_d['date']).astype(int) / 10**9).astype(int)))
# df = (pd.read_csv('data/raw/lol/2023_LoL_esports_match_data_from_OraclesElixir.csv')
#       .assign(ts = lambda _d: (pd.to_datetime(_d['date']).astype(int) / 10**9).astype(int)))

/tmp/ipykernel_26068/2767207795.py:13: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = (pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
/tmp/ipykernel_26068/2767207795.py:13: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = (pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)


In [2]:
home = df.loc[df['side'] == 'Blue', ['teamid', 'gameid', 'result', 'ts', 'gamelength']].dropna().drop_duplicates()
away = df.loc[df['side'] == 'Red', ['teamid', 'gameid']].dropna().drop_duplicates()

In [3]:
matches = home.merge(away, how='inner', on='gameid')
players = df.loc[df['gameid'].isin(matches['gameid'].unique()), ['teamid', 'playerid', 'ts']].dropna().drop_duplicates()

In [4]:
matches = (matches
           .assign(ts = lambda _d: _d['ts'] + _d['gamelength'])
           .rename(columns={'teamid_x': 'u', 'teamid_y': 'v', 'result': 'e_type'})
           .assign(e_type = lambda _d: _d['e_type'] + 1)
           .assign(u_type = 1)
           .assign(v_type = 1)
           [['u', 'v', 'ts', 'e_type', 'u_type', 'v_type']]
           .reset_index(drop=True))

players = (players
           .assign(e_type = 3)
           .assign(u_type = 1)
           .assign(v_type = 2)
           .rename(columns={'teamid': 'u', 'playerid': 'v'})
           .reset_index(drop=True))

In [5]:
teams = pd.concat([matches['u'], matches['v']]).sort_values().unique()
idx_teams = np.arange(len(teams))
teams_dict = {teams[k]: k for k in idx_teams}

player = players['v'].sort_values().unique()
idx_player = np.arange(len(player))
player_dict = {player[k]: k + len(teams) for k in idx_player}

In [6]:
matches = (matches
           .assign(u = lambda _d: _d['u'].map(lambda x: teams_dict[x]))
           .assign(v = lambda _d: _d['v'].map(lambda x: teams_dict[x])))

players = (players
           .assign(u = lambda _d: _d['u'].map(lambda x: teams_dict[x]))
           .assign(v = lambda _d: _d['v'].map(lambda x: player_dict[x])))

In [7]:
events = pd.concat([matches, players]).sort_values('ts').reset_index(drop=True)

In [8]:
NUM_NODE = len(player) + len(teams)
NUM_EV = len(events)
NUM_N_TYPE = 2
NUM_E_TYPE = 3

In [9]:
events = (events
          .assign(e_idx = np.arange(1, NUM_EV + 1)))

In [10]:
print("num node:", NUM_NODE)
print("num events:", NUM_EV)
events.to_csv(OUT_EV, index=None)
desc = {
        "num_node": NUM_NODE,
        "num_edge": NUM_EV,
        "num_node_type": NUM_N_TYPE,
        "num_edge_type": NUM_E_TYPE
    }
with open(OUT_DESC, 'w') as f:
    json.dump(desc, f, indent=4)

num node: 10527
num events: 783680
